In [1]:
import pandas as pd

In [ ]:
data_directory = '../raw/'

school_df = pd \
    .read_csv( f'{data_directory}/pubschls.txt', sep = '\t' ) \
    [lambda _df: _df['County'] == 'San Francisco'] \
    [lambda _df: _df['School'] != 'No Data']

enrollment_df = pd.DataFrame()
data_filenames = [ f'enr{yyyyyy}-v2.txt' for yyyyyy in [ '201416', '201719', '202022' ] ]
for data_filename in data_filenames:
    enrollment_df = pd.concat(
        [
            enrollment_df,
            pd
                .read_csv( f'{data_directory}/{data_filename}', sep = '\t' )
                .merge( school_df[['CDSCode','School']], left_on = ['CDS_CODE'], right_on = ['CDSCode'] )
        ],
        axis = 0,
        ignore_index = True
    )

In [8]:
school_df

,CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,...,Virtual,Magnet,YearRoundYN,FederalDFCDistrictID,Latitude,Longitude,AdmFName,AdmLName,LastUpDate,Multilingual
13493,38103893830114,0691111,No Data,Closed,San Francisco,San Francisco County Office of Education,Bay Junior/Senior High,Building 141 Fort Mason,Bldg. 141 Fort Mason,San Francisco,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,06/24/1999,No Data
13494,38103893830304,0691111,No Data,Merged,San Francisco,San Francisco County Office of Education,Log Cabin Ranch School,La Honda,La Honda,La Honda,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,06/24/1999,No Data
13495,38103893830312,0691111,No Data,Merged,San Francisco,San Francisco County Office of Education,Postal Street Academy,914 Divisadero,914 Divisadero,San Francisco,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,06/24/1999,No Data
13496,38103893830320,0691111,No Data,Merged,San Francisco,San Francisco County Office of Education,Andrew Jackson Shelter School,2049 Grove Street,2049 Grove St.,San Francisco,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,06/24/1999,No Data
13497,38103893830338,0691111,No Data,Merged,San Francisco,San Francisco County Office of Education,County Community Day School,1512 Golden Gate Avenue,1512 Golden Gate Ave.,San Francisco,...,No Data,No Data,N,No Data,No Data,No Data,No Data,No Data,06/24/1999,No Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13749,38767030121814,0601411,12672,Closed,San Francisco,SBE - San Francisco Flex Academy,San Francisco Flex Academy,1350 7th Avenue,1350 7th Ave.,San Francisco,...,N,N,Y,No Data,37.763243,-122.46382,No Data,No Data,07/28/2016,No Data
13752,38769190132159,No Data,13938,Closed,San Francisco,SBE - OnePurpose,OnePurpose,948 Hollister Avenue,948 Hollister Ave.,San Francisco,...,N,N,N,No Data,37.718332,-122.38988,No Data,No Data,07/01/2020,N
13754,38769270132183,No Data,13939,Active,San Francisco,SBE - The New School of San Francisco,The New School of San Francisco,940 Filbert St,940 Filbert St,San Francisco,...,N,N,N,0602439,37.801011,-122.41509,Emily,Bobel Kilduff,10/09/2024,N
13756,38771310137307,No Data,14322,Active,San Francisco,SBE - KIPP Bayview Elementary,KIPP Bayview Elementary,948 Hollister Avenue,948 Hollister Ave.,San Francisco,...,N,N,Y,0601498,37.718359,-122.38949,Jess,Wood,09/09/2025,N
